In [ ]:
from __future__ import print_function, division
from sympy import *
import ast
import numpy as np
import matplotlib.pyplot as plt
#from brokenaxes import brokenaxes
#from numba import jit

In [ ]:
from matplotlib import rc
rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
rc('text', usetex=True)

In [ ]:
name = "free_energy_def.dat"
tmp = open(name, "r")
tmp=tmp.read()
exec(tmp)

In [ ]:
tmp

In [ ]:
name = "assign_symbols_final.dat"
tmp = open(name, "r")
tmp=tmp.read()
exec(tmp)

In [ ]:
w_array = np.loadtxt("w_array.dat")

In [ ]:
len(w_array[0])

In [ ]:
name = "ML_parameter.dat" #load n_conv, con_dim,kill (killsmall)
tmp = open(name, "r")
tmp=tmp.read()
exec(tmp)
kill=float(kill)
print(n_parameter,conv_dim,n_conv,kill)

In [ ]:
data_file = '../LJ_data_wall/data_all/'
f=np.loadtxt(data_file+"MC_parameter.dat")
L = float(f[0])
dx = float(f[1])
N = int(L/dx)
print(N,L,dx)

In [ ]:
#for i in range (len(w_array)):
#    w_array[i][0:int(2//dx)]=0
#    w_array[i][conv_dim-int(2//dx):conv_dim]=0
    

In [ ]:
x=np.linspace(-conv_dim*dx/2,conv_dim*dx/2,conv_dim)
for i in range (n_conv):
    plt.plot(x,w_array[i],linewidth=3,label=r"$\omega_{"+str(i)+"}$")
plt.legend(fontsize=15)
plt.xlabel(r"$x(\sigma)$",fontsize=20)
plt.ylabel(r"$\omega$",fontsize=20)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
#plt.xlim([-4,4])
#plt.ylim([np.min(w_array)-0.05,0.8])

plt.savefig("whr.eps",format="eps",bbox_inches='tight')

In [ ]:
fed_eqn

In [ ]:
fed_eqn()

In [ ]:
name = "all_parameter.dat"
tmp = open(name, "r")
tmp=tmp.read()
#print(tmp)
tmp=tmp.replace("/varible:0",":")
tmp=tmp.replace("/const_1:0",":")

for i in range(100):
    tmp=tmp.replace("_"+str(i),"")
tmp=tmp.replace("[[","")
tmp=tmp.replace("]]","")
tmp=tmp.replace("\n",",")
tmp=tmp.replace(":","\":")
tmp=tmp.replace(",",",\"")
tmp="{\""+tmp+"}"
tmp=tmp.replace(",\"}","}")
#print(tmp)
dict_all=ast.literal_eval(tmp)

In [ ]:
dict_all

In [ ]:
kill_small=0
for i in list(dict_all.keys()):
    if(np.abs(dict_all[i])<kill and i[0]=="a"):
        dict_all[i]=0
        kill_small+=1
print(len(dict_all.keys()),kill_small)

In [ ]:
input_symbols = fed_eqn().subs(dict_all).free_symbols

In [ ]:
 fed_eqn().subs(dict_all)

In [ ]:
input_str = "["
for i in range(n_parameter):
    input_str+="seps"+str(i)+","
for i in range(n_conv):
    input_str+="sn"+str(i)+","
input_str += "]"
tmp = "input_str="+input_str
exec(tmp)

In [ ]:
[input_str]

In [ ]:
fd = lambdify(input_str,fed_eqn().subs(dict_all))

In [ ]:
fd

In [ ]:
#simplify(diff(fed_eqn().subs(dict_all),sn0)-diff(fed_eqn(),sn0).subs(dict_all))

In [ ]:
for i in range(n_conv):
    eqn_str = "fd_n"+str(i)+" =lambdify(input_str,diff(fed_eqn(),sn"+str(i)+").subs(dict_all))"
    print(eqn_str)
    exec(eqn_str)

In [ ]:
fd

In [ ]:
fd_n2

In [ ]:
n_array = np.zeros([n_conv,N])

In [ ]:
#@jit
def conv (rho,w):
    LLw = int((conv_dim-1)/2)
    #print(LLw)
    rho_pad=np.copy(rho)
    rho_tile=np.tile(rho_pad,3)
    rho_pad=rho_tile[N-LLw:2*N+LLw]
    n = np.correlate(rho_pad,w)
    return n*dx

# $C_2$

In [ ]:
def cal_c2_FMT(x,rho0):
    c2 = np.zeros(x.shape[0])
    for i in range(Lw):
        r=np.abs(x[i])
        eta = rho0
        if(r<1):
            c2[i]=-1/(1-eta)-eta/((1-eta)**2)*(1-r)
        else:
            c2[i]=0
    return c2
    

In [ ]:
def wconvw (rho,w):
    Lw=conv_dim
    LLw = int((conv_dim-1)/2)
    rho_pad=np.copy(rho)
    rho_pad=np.pad(rho_pad,(LLw,LLw),mode='constant',constant_values=0)
    n = np.correlate(rho_pad,w)
    return n*dx

In [ ]:
fd

In [ ]:
c2_sim=np.loadtxt("../c2_LJ/c2_z_1.5_eps_1.5_rho_avg_0.61")

In [ ]:
def cal_c2_ML(rho0,eps):
    srho0=symbols('rho') 
    
    sub_str = "({"
    for i in range (n_conv):
        sub_str+="\"n"+str(i)+"\":np.sum(w_array["+str(i)+"])*dx*srho0,"
    sub_str += "seps0:"+str(eps)
    sub_str += "})"
    fd=fed_eqn().subs(dict_all)
    c2 = np.zeros(conv_dim)
    for i in range (n_conv):
        for j in range (n_conv):
            print(i,j,n_conv)
            temp=wconvw(w_array[i],w_array[j])
            str_temp= "diff(diff(fd,sn"+str(i)+"),sn"+str(j)+").subs"+sub_str
            tmp=eval(str_temp)
            tmp=float(tmp.subs({"rho":rho0}))
            #tmp=float(tmp.subs({"eps":eps}))
            
            #print(type(tmp),tmp)
            #print(type(tmp),tmp)
            
            #print(tmp.subs({"rho":rho0}))
            #print(tmp,type(Float(tmp)),type(temp))
            c2-=eps*tmp*temp
    return c2

In [ ]:
c2=cal_c2_ML(0.5,0.5)

In [ ]:
Lw=conv_dim
LLw = int((conv_dim-1)/2)
rho0=0.5
x = np.linspace(-LLw,LLw,Lw)*dx
plt.plot(x,cal_c2_ML(rho0,0.5),"k--",label="ML")
plt.legend(fontsize=20)

plt.ylabel(r"$C_2(x)$",fontsize=25)
plt.xlabel(r"$x(\sigma)$",fontsize=25)
plt.title(r"$\rho="+str(rho0)+"$",fontsize=25)

In [ ]:
Lw=conv_dim
LLw = int((conv_dim-1)/2)

x = np.linspace(-LLw,LLw,Lw)*dx


c2_sim=np.loadtxt("../c2_LJ/c2_z_1.85_eps_1.85_rho_avg_0.70")
eps=1.85
rho0=0.703
plt.plot(c2_sim[0],c2_sim[1],"k-",lw=3,label=r"MC,\,$\rho_0=$"+str(rho0)+ r"$\,,\epsilon$="+str(eps))
plt.plot(x,cal_c2_ML(rho0,eps)+cal_c2_FMT(x,rho0),"k--",lw=3,label="ML")
plt.legend(fontsize=12,loc="best")

plt.ylabel(r"$C_2(x)$",fontsize=25)
plt.xlabel(r"$x(\sigma)$",fontsize=25)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
#plt.title(r"$\rho="+str(rho0)+"$",fontsize=25)
plt.xlim(0,3)



plt.savefig("c2_LJ.eps",format="eps",bbox_inches='tight')

In [ ]:
Lw=conv_dim
LLw = int((conv_dim-1)/2)
rho0=0.1
x = np.linspace(-LLw,LLw,Lw)*dx
plt.plot(x,cal_c2_ML(rho0,0.1)+cal_c2_FMT(x,rho0),"k--",label="ML")
plt.legend(fontsize=20)

plt.ylabel(r"$C_2(x)$",fontsize=25)
plt.xlabel(r"$x(\sigma)$",fontsize=25)
plt.title(r"$\rho="+str(rho0)+"$",fontsize=25)

In [ ]:

x = np.linspace(-LLw,LLw,Lw)*dx

c2_sim=np.loadtxt("../c2_LJ/c2_z_1.85_eps_1.85_rho_avg_0.70")
c2_ML_nosplitting=np.loadtxt(
    "../LJ_no_split/ML_no_splitting_c2_eps_1.85_rho0_0.703.dat")

eps=1.85
rho0=0.7


plt.plot(c2_sim[0],c2_sim[1],"k-",lw=3,label=r"LJ, MC")
plt.plot(c2_ML_nosplitting[0][::2],c2_ML_nosplitting[1][::2],"grey",linestyle='dotted',lw=3,label="LJ, ML-no splitting")
plt.plot(x,cal_c2_ML(rho0,eps)+cal_c2_FMT(x,rho0),"red",linestyle='dashed',lw=3,label="LJ, ML-splitting")

c2_ML_HR=np.loadtxt(
    "../../HR/functional_test/ML_HR_rho0_0.70.dat")
#plt.plot(x,cal_c2_FMT(x,0.7),"b-",lw=2,label=r"HR, FMT,\,$\rho_0=$"+str(rho0))
plt.plot(x,cal_c2_FMT(x,0.7),"b-",lw=3,label=r"HR, Percus")
plt.plot(c2_ML_HR[0],c2_ML_HR[1],"g--",lw=3,label=r"HR, ML")




plt.legend(fontsize=12,loc="best")
plt.ylabel(r"$C^{(2)}(x)$",fontsize=25)
plt.xlabel(r"$x/\sigma$",fontsize=25)
plt.xticks(fontsize=20)
plt.yticks(np.linspace(-20,8,8),fontsize=20)
#plt.title(r"$\rho="+str(rho0)+"$",fontsize=25)
plt.xlim(0,2.7)
#plt.xlim(-4,4)

plt.ylim(-12,4.5)

plt.savefig("c2_all.eps",format="eps",bbox_inches='tight')